# Model Training

In [1]:
from google.colab import drive
drive.mount('/content/drive')
# Python program to create Image Classifier using CNN 
#  ________________________________________________________________
import cv2 
import os 
import numpy as np 
from random import shuffle 
from tqdm import tqdm 
import random



'''Setting up the env'''

TRAIN_DIR = '/content/drive/MyDrive/train'
TEST_DIR = '/content/drive/MyDrive/test'
IMG_SIZE = 30

CATEGORIES = ['cats', 'dogs']
train_data = []
test_data = []

for category in CATEGORIES:
    path = os.path.join(TRAIN_DIR, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        label = CATEGORIES.index(category)
        arr = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        new_arr = cv2.resize(arr, (IMG_SIZE, IMG_SIZE))
        train_data.append([new_arr, label])

for img in os.listdir(TEST_DIR):
  img_path = os.path.join(TEST_DIR, img)
  img_num = img.split('.')[0] 
  arr = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
  new_arr = cv2.resize(arr, (IMG_SIZE, IMG_SIZE))
  test_data.append([new_arr, img_num])

random.shuffle(train_data)
random.shuffle(test_data)

train_X = []
train_Y = []
for features, label in train_data:
    train_X.append(features)
    train_Y.append(label)

train_X = np.array(train_X) / 255
train_Y = np.array(train_Y)
train_X = train_X.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
#from keras.losses import SparseCategoricalCrossentropy
import tensorflow as tf

model = Sequential()

model.add(Conv2D(64, (7,7), activation = 'relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64, (5,5), activation = 'relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())
model.add(Dense(8, input_shape = train_X.shape[1:], activation = 'sigmoid'))
model.add(Dense(2, activation = 'sigmoid'))
#model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

model.fit(train_X, train_Y, epochs = 20, validation_split=0.3)

Mounted at /content/drive
Epoch 1/20


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


504/504 [==============================] - 37s 71ms/step - loss: 0.6807 - accuracy: 0.5487 - val_loss: 0.6664 - val_accuracy: 0.5866
Epoch 2/20
504/504 [==============================] - 36s 72ms/step - loss: 0.6487 - accuracy: 0.6201 - val_loss: 0.6474 - val_accuracy: 0.6341
Epoch 3/20
504/504 [==============================] - 36s 71ms/step - loss: 0.6248 - accuracy: 0.6533 - val_loss: 0.6398 - val_accuracy: 0.6287
Epoch 4/20
504/504 [==============================] - 36s 71ms/step - loss: 0.6027 - accuracy: 0.6721 - val_loss: 0.6093 - val_accuracy: 0.6729
Epoch 5/20
504/504 [==============================] - 36s 71ms/step - loss: 0.5840 - accuracy: 0.6933 - val_loss: 0.6102 - val_accuracy: 0.6741
Epoch 6/20
504/504 [==============================] - 36s 71ms/step - loss: 0.5703 - accuracy: 0.7041 - val_loss: 0.6048 - val_accuracy: 0.6747
Epoch 7/20
504/504 [==============================] - 36s 71ms/step - loss: 0.5499 - accuracy: 0.7181 - val_loss: 0.5936 - val_accuracy: 0.6902
Epo

# Weights Extraction

In [2]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 64)        3200      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 64)          102464    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 4, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 2, 2, 64)          36928     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 1, 1, 64)         0

In [3]:
# Layer 0-1
WeightsLayer01 = np.transpose(model.layers[7].get_weights()[0])
BiasesLayer01 = model.layers[7].get_weights()[1]
#print(WeightsLayer01)

In [4]:
# Layer 1-2
WeightsLayer12 = np.transpose(model.layers[8].get_weights()[0])
BiasesLayer12 = model.layers[8].get_weights()[1]
#print(WeightsLayer12)

#Weights Conversion from decimal to floating point representation to hexadecimal and saving to .txt


In [5]:
def DecimalToFloatPt32_hex_txt(matrix, name):
  #### Decimal to single precision floating point representation
  import struct
  def floatToBinary32(value):
      return ''.join(f'{c:0>8b}' for c in struct.pack('!f', value))

  matrix_FloatPt32_binary = []
  for row in matrix:
    temp=[]
    for values in row:
    #print(values)
    #break
      temp.append(floatToBinary32(values))
    matrix_FloatPt32_binary.append(temp)


  #### single precision floating point representation to Hexadecimal
  matrix_FloatPt32_hex = []
  for row in matrix_FloatPt32_binary:
    temp = []
    for values in row:
      temp.append("{0:0>4X}".format(int(values, 2)))
    matrix_FloatPt32_hex.append(temp)

  #### writes each weight in each new line
  file = open(name, "w+")
  for row in matrix_FloatPt32_hex:
    for values in row:
      file.write(values)
      file.write("\n")
  file.close()

# Weights of Layer 0-1
DecimalToFloatPt32_hex_txt(WeightsLayer01, "WeightsLayer01_FloatPt32_hex.txt")

In [6]:
# Bias of Layer 0-1
#### Decimal to single precision floating point representation
import struct
def floatToBinary32(value):
    return ''.join(f'{c:0>8b}' for c in struct.pack('!f', value))

BiasesLayer01_FloatPt32_binary = []
for row in BiasesLayer01:
  BiasesLayer01_FloatPt32_binary.append(floatToBinary32(row))


#### single precision floating point representation to Hexadecimal
BiasesLayer01_FloatPt32_hex = []
for row in BiasesLayer01_FloatPt32_binary:
  BiasesLayer01_FloatPt32_hex.append("{0:0>4X}".format(int(row, 2)))

#### writes each weight in each new line
file = open("BiasesLayer01_FloatPt32_hex.txt", "w+")
for row in BiasesLayer01_FloatPt32_hex:
  file.write(row)
  file.write("\n")
file.close()

In [7]:
# Weights of Layer 1-2
DecimalToFloatPt32_hex_txt(WeightsLayer12, "WeightsLayer12_FloatPt32_hex.txt")

In [8]:
# Bias of Layer 1-2
#### Decimal to single precision floating point representation
import struct
def floatToBinary32(value):
    return ''.join(f'{c:0>8b}' for c in struct.pack('!f', value))

BiasesLayer12_FloatPt32_binary = []
for row in BiasesLayer12:
  BiasesLayer12_FloatPt32_binary.append(floatToBinary32(row))


#### single precision floating point representation to Hexadecimal
BiasesLayer12_FloatPt32_hex = []
for row in BiasesLayer12_FloatPt32_binary:
  BiasesLayer12_FloatPt32_hex.append("{0:0>4X}".format(int(row, 2)))

#### writes each weight in each new line
file = open("BiasesLayer12_FloatPt32_hex.txt", "w+")
for row in BiasesLayer12_FloatPt32_hex:
  file.write(row)
  file.write("\n")
file.close()

#From .txt file to .mif format

In [9]:
'''
This script helps to convert hex instructions to Quartus's .mif files.
To run it, use "python hex2mif.py input.txt output.mif"
For input file, assuming you are using Mars as assember,
you need to export the binary instruction as hexadecimal text file.
But for any kind of input files, it should work fine as long as
it uses the following format:
DEADBEEF
BAADF00D
'''

import sys

def read(f_in):
    with open(f_in) as f:
        return [int(i, 16) for i in f if len(i) != 0]


def write(f_out, data, width=32, depth=65536):
    if len(data) > depth:
        print('Data larger than memory size, abort.')
    else:
        buf = 'WIDTH={:d};\nDEPTH={:d};\nADDRESS_RADIX=HEX;\nDATA_RADIX=HEX;\nCONTENT BEGIN\n'.format(width, depth)
        l_index = str(len('{:x}'.format(depth)))
        l_data = str(int(width / 4))
        s = '\t{:0' + l_index + 'X} : {:0' + l_data + 'X};\n'
        for i, j in enumerate(data):
            buf += s.format(i, j)
        if len(buf) == depth - 1:
            buf += s.format(depth - 1, 0)
        elif len(buf) < depth - 1:
            buf += ('\t[{:0' + l_index + 'X}..{:0' + l_index + 'X}] : {:0' + l_data + 'X};\n').format(len(data), depth - 1, 0)
        buf += 'END;\n'
        with open(f_out, 'w') as f:
            f.write(buf)


def convert(f_in, f_out):
    write(f_out, read(f_in))



In [10]:
#Weights
convert("WeightsLayer01_FloatPt32_hex.txt", "WeightsLayer01_FloatPt32_hex.mif")
convert("WeightsLayer12_FloatPt32_hex.txt", "WeightsLayer12_FloatPt32_hex.mif")

#Bias
convert("BiasesLayer01_FloatPt32_hex.txt", "BiasesLayer01_FloatPt32_hex.mif")
convert("BiasesLayer12_FloatPt32_hex.txt", "BiasesLayer12_FloatPt32_hex.mif")

#Input Image Conversion from decimal to floating point representation to hexadecimal and saving to .txt

In [11]:

for i in range(10):
  from keras import backend as K
  # with a Sequential model
  get_3rd_layer_output = K.function([model.layers[0].input], [model.layers[6].output])
  layer_output = get_3rd_layer_output([train_data[i][0].reshape(-1,IMG_SIZE, IMG_SIZE, 1) ])[0]
  #print(layer_output)


  #### Decimal to single precision floating point representation
  import struct
  def floatToBinary32(value):
      return ''.join(f'{c:0>8b}' for c in struct.pack('!f', value))

  ip_FloatPt32_binary = []
  for row in layer_output[0]:
    ip_FloatPt32_binary.append(floatToBinary32(row))


  #### single precision floating point representation to Hexadecimal
  ip_FloatPt32_hex = []
  for row in ip_FloatPt32_binary:
    ip_FloatPt32_hex.append("{0:0>4X}".format(int(row, 2)))

  #### writes each weight in each new line
  file = open("ip"+str(i+1)+".txt", "w+")
  for row in ip_FloatPt32_hex:
    file.write(row)
    file.write("\n")
  file.close()

#From .txt file to .mif format

In [12]:
'''
This script helps to convert hex instructions to Quartus's .mif files.
To run it, use "python hex2mif.py input.txt output.mif"
For input file, assuming you are using Mars as assember,
you need to export the binary instruction as hexadecimal text file.
But for any kind of input files, it should work fine as long as
it uses the following format:
DEADBEEF
BAADF00D
'''

import sys

def read(f_in):
    with open(f_in) as f:
        return [int(i, 16) for i in f if len(i) != 0]


def write(f_out, data, width=32, depth=65536):
    if len(data) > depth:
        print('Data larger than memory size, abort.')
    else:
        buf = 'WIDTH={:d};\nDEPTH={:d};\nADDRESS_RADIX=HEX;\nDATA_RADIX=HEX;\nCONTENT BEGIN\n'.format(width, depth)
        l_index = str(len('{:x}'.format(depth)))
        l_data = str(int(width / 4))
        s = '\t{:0' + l_index + 'X} : {:0' + l_data + 'X};\n'
        for i, j in enumerate(data):
            buf += s.format(i, j)
        if len(buf) == depth - 1:
            buf += s.format(depth - 1, 0)
        elif len(buf) < depth - 1:
            buf += ('\t[{:0' + l_index + 'X}..{:0' + l_index + 'X}] : {:0' + l_data + 'X};\n').format(len(data), depth - 1, 0)
        buf += 'END;\n'
        with open(f_out, 'w') as f:
            f.write(buf)


def convert(f_in, f_out):
    write(f_out, read(f_in))

for i in range(10):
  convert("ip"+str(i+1)+".txt", "ip"+str(i+1)+".mif")